## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-16-07-48-00 +0000,wsj,Indonesia’s Central Bank Cuts Rates to Bolster...,https://www.wsj.com/articles/indonesias-centra...
1,2025-07-16-07-46-29 +0000,nyt,Iceland Volcano Erupts Again but Doesn’t Threa...,https://www.nytimes.com/2025/07/16/world/europ...
2,2025-07-16-07-46-18 +0000,bbc,Co-op boss says sorry to 6.5m people who had d...,https://www.bbc.com/news/articles/cql0ple066po
3,2025-07-16-07-29-42 +0000,bbc,UK inflation hits highest for almost a year an...,https://www.bbc.com/news/articles/c3en2enpy7po
4,2025-07-16-07-24-41 +0000,bbc,Rare Gandhi oil portrait sold for more than $2...,https://www.bbc.com/news/articles/cjd2gx9l8kro


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2313/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
38,trump,26
47,new,6
22,inflation,6
43,up,5
132,administration,5


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
62,2025-07-15-19-03-00 +0000,wsj,JPMorgan CEO Jamie Dimon sounded Wall Street’s...,https://www.wsj.com/economy/central-banking/di...,49
33,2025-07-16-00-00-00 +0000,wsj,"President Trump wants tariffs, the higher the ...",https://www.wsj.com/economy/trade/forget-taco-...,48
26,2025-07-16-01-00-00 +0000,wsj,The impact of President Trump’s whirlwind six ...,https://www.wsj.com/economy/trump-effect-start...,45
64,2025-07-15-18-44-15 +0000,nyt,Rising Inflation Underscores Risks in Trump’s ...,https://www.nytimes.com/2025/07/15/us/politics...,45
86,2025-07-15-11-15-55 +0000,nyt,E.U. Nations Plan Retaliatory Tariffs Against ...,https://www.nytimes.com/2025/07/15/world/europ...,42


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
62,49,2025-07-15-19-03-00 +0000,wsj,JPMorgan CEO Jamie Dimon sounded Wall Street’s...,https://www.wsj.com/economy/central-banking/di...
7,29,2025-07-16-06-35-00 +0000,wsj,U.K. Inflation Heats Up as Bank of England Sta...,https://www.wsj.com/economy/u-k-inflation-heat...
33,22,2025-07-16-00-00-00 +0000,wsj,"President Trump wants tariffs, the higher the ...",https://www.wsj.com/economy/trade/forget-taco-...
57,20,2025-07-15-20-00-01 +0000,nyt,House Democrats Push for Hearing on Jeffrey Ep...,https://www.nytimes.com/2025/07/15/us/politics...
28,19,2025-07-16-00-23-39 +0000,bbc,Thousands of Afghans were moved to UK in secre...,https://www.bbc.com/news/articles/cvg8zy78787o
11,17,2025-07-16-05-37-50 +0000,bbc,Driver held for hit-and-run death of world's '...,https://www.bbc.com/news/articles/c2d0yep9d37o
67,16,2025-07-15-18-11-29 +0000,bbc,French PM looks to scrap two public holidays i...,https://www.bbc.com/news/articles/c2d02dz6gedo
23,14,2025-07-16-01-22-14 +0000,nyt,Republicans in Congress Shift to Backing Ukrai...,https://www.nytimes.com/2025/07/15/us/politics...
38,14,2025-07-15-23-10-07 +0000,nyt,Canada’s Mark Carney Says There’s Little Hope ...,https://www.nytimes.com/2025/07/15/us/politics...
89,14,2025-07-15-10-38-33 +0000,nyt,Blazes in Northern Ireland Recall an Old Messa...,https://www.nytimes.com/2025/07/15/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
